In [1]:
# Native imports
import os
import json

# Import .env into loaded environment variables
from dotenv import load_dotenv
load_dotenv('../sc-backend/.env.dev')

# MongoDB Python driver import
import pymongo

# 3rd-party data science imports
import pandas as pd
import seaborn as sns
pd.set_option('display.max_rows', None)

OUTPUT_ORG_INFO_FILE = 'org-info.json'

# Step 0: Load the data

In [2]:
with open(OUTPUT_ORG_INFO_FILE, 'r') as fp:
    club_info_json = json.load(fp)
club_json_df = pd.DataFrame(club_info_json)
club_json_df = club_json_df[['name', 'email']]
club_json_df['name'].apply(lambda s: s if pd.isna(s) else s.strip())
club_json_df['email'].apply(lambda s: s if pd.isna(s) else s.lower().strip())
club_json_df.head()

,name,email
0,180 Degrees Consulting at Berkeley,berkeley@180dc.org
1,3D Modeling and Animation at Berkeley,3dma.at.berkeley@gmail.com
2,5&2 Community Service,fiveandtwo.berkeley@gmail.com
3,A Better Way at Berkeley,victorjann@berkeley.edu
4,ABA,tiffany.s.y.chen@berkeley.edu


In [3]:
club_json_df_copy = club_json_df.copy()
club_json_df_copy = club_json_df_copy[~pd.isna(club_json_df_copy['email'])]
club_json_df_copy.to_csv('callink-club-info.csv')
print(len(club_json_df_copy))

1024


In [15]:
conn_string = os.getenv('MONGO_URI')
client = pymongo.MongoClient(conn_string)

In [5]:
club_info_db = []
for club in client['develop-db']['future_user'].find():
    club_info_db += [{'name': club['org_name'].strip(), 'email': club['org_email'].lower().strip()}]
club_db_df = pd.DataFrame(club_info_db)
club_db_df.head()

,name,email
0,Phi Alpha Theta,phat.berkeley@gmail.com
1,Berkeley Engineers and Mentors--BEAM,beam.teach@gmail.com
2,Phoenix Consulting Group,nian_s@berkeley.edu
3,Product Space at UC Berkeley,cal@productspace.org
4,The Berkeley Alt. Protein Project,berkeleyaltprotein@gmail.com


In [6]:
existing_clubs_df = pd.merge(club_json_df, club_db_df, how='inner', on='email')
existing_clubs_df.head()

,name_x,email,name_y
0,5&2 Community Service,fiveandtwo.berkeley@gmail.com,5&2 Berkeley
1,Abba Modern,abbamodern@gmail.com,Abba Modern
2,acts2fellowship,info@acts2fellowship.org,acts2fellowship
3,AFX Dance,afxdance@gmail.com,AFX Dance
4,Alternative Breaks,berkeleyaltbreaks@gmail.com,Alternative Breaks


In [7]:
signed_up_clubs_in_callink_percent = len(existing_clubs_df) / len(club_db_df) * 100
print('# of clubs on CalLink:', len(club_json_df))
print('# of clubs on Sproul.club:', len(club_db_df))
print()
print('# of clubs on both platforms:', len(existing_clubs_df))
print('%% of signed up clubs within CalLink: %.1f%%' % signed_up_clubs_in_callink_percent)

# of clubs on CalLink: 1309
# of clubs on Sproul.club: 199

# of clubs on both platforms: 124
% of signed up clubs within CalLink: 62.3%


# Step 1a: Merge data by same email

In [8]:
existing_clubs_df_by_email = pd.merge(
    club_json_df, club_db_df,
    how='inner', on='email',
    suffixes=('_callink', '_mongo')
)

matching_names = existing_clubs_df_by_email['name_callink'] == existing_clubs_df_by_email['name_mongo']
existing_clubs_df_by_email['match'] = matching_names
existing_clubs_df_by_email

,name_callink,email,name_mongo,match
0,5&2 Community Service,fiveandtwo.berkeley@gmail.com,5&2 Berkeley,False
1,Abba Modern,abbamodern@gmail.com,Abba Modern,True
2,acts2fellowship,info@acts2fellowship.org,acts2fellowship,True
3,AFX Dance,afxdance@gmail.com,AFX Dance,True
4,Alternative Breaks,berkeleyaltbreaks@gmail.com,Alternative Breaks,True
5,American Chemical Society at Berkeley Student ...,acsberkeley@gmail.com,American Chemical Society at Berkeley (ACS@Ber...,False
6,American Medical Student Association--Berkeley...,berkeleyamsa@gmail.com,American Medical Student Association,False
7,Artists in Resonance,artistsinresonanceucb@gmail.com,Artists in Resonance,True
8,"Asha for Education, Berkeley",berkeley@ashanet.org,"Asha for Education, Berkeley",True
9,Asian American and Pacific Islander Health Res...,aapihrg.info@gmail.com,Asian American and Pacific Islander Health Res...,True


In [9]:
print('# of matching names:', sum(matching_names))
print('# of non-matching names:', len(matching_names) - sum(matching_names))
print('%% of matching names: %.1f%%' % ( sum(matching_names) / len(matching_names) * 100 ))

# of matching names: 68
# of non-matching names: 56
% of matching names: 54.8%


# Step 1b: Merge data by same name

In [10]:
existing_clubs_df_by_name = pd.merge(
    club_json_df, club_db_df,
    how='inner', on='name',
    suffixes=('_callink', '_mongo')
)

matching_emails = existing_clubs_df_by_name['email_callink'] == existing_clubs_df_by_name['email_mongo']
existing_clubs_df_by_name['match'] = matching_emails
existing_clubs_df_by_name

,name,email_callink,email_mongo,match
0,3D Modeling and Animation at Berkeley,3dma.at.berkeley@gmail.com,staff@ucbugg.com,False
1,Abba Modern,abbamodern@gmail.com,abbamodern@gmail.com,True
2,acts2fellowship,info@acts2fellowship.org,info@acts2fellowship.org,True
3,AFX Dance,afxdance@gmail.com,afxdance@gmail.com,True
4,Alpha Kappa Psi,sanampatel1999@gmail.com,corporaterelations@calakpsi.com,False
5,Alternative Breaks,berkeleyaltbreaks@gmail.com,berkeleyaltbreaks@gmail.com,True
6,Artists in Resonance,artistsinresonanceucb@gmail.com,artistsinresonanceucb@gmail.com,True
7,"Asha for Education, Berkeley",berkeley@ashanet.org,berkeley@ashanet.org,True
8,Asian American and Pacific Islander Health Res...,aapihrg.info@gmail.com,aapihrg.info@gmail.com,True
9,Asian Pacific American Pre-Law Association,ucb.apapla@gmail.com,ucb.apapla@gmail.com,True


In [11]:
print('# of matching emails:', sum(matching_emails))
print('# of non-matching emails:', len(matching_emails) - sum(matching_emails))
print('%% of matching emails: %.1f%%' % ( sum(matching_emails) / len(matching_emails) * 100 ))

# of matching emails: 68
# of non-matching emails: 29
% of matching emails: 70.1%


# Step 1c: Merge data by same email and name

In [12]:
existing_clubs_df_by_email_name = pd.merge(
    club_json_df, club_db_df,
    how='inner',
    suffixes=('_callink', '_mongo')
)

existing_clubs_df_by_email_name

,name,email
0,Abba Modern,abbamodern@gmail.com
1,acts2fellowship,info@acts2fellowship.org
2,AFX Dance,afxdance@gmail.com
3,Alternative Breaks,berkeleyaltbreaks@gmail.com
4,Artists in Resonance,artistsinresonanceucb@gmail.com
5,"Asha for Education, Berkeley",berkeley@ashanet.org
6,Asian American and Pacific Islander Health Res...,aapihrg.info@gmail.com
7,Asian Pacific American Pre-Law Association,ucb.apapla@gmail.com
8,ASUC Mental Health Commission,mentalhealthcommission@asuc.org
9,ASUC Student Legal Clinic,asuclegalclinic@gmail.com


In [13]:
print('# of clubs matching by name and email:', len(existing_clubs_df_by_email_name))

# of clubs matching by name and email: 68
